In [11]:
from langchain.tools import Tool
from langchain_community.llms import LlamaCpp
from langchain_core.messages import HumanMessage, SystemMessage
from typing import Dict, Any, List, Tuple, TypedDict
from langgraph.graph import Graph, StateGraph
import json

# Initialize Mistral model
mistral = LlamaCpp(
    model_path="C:/Users/DaysPC/Langgraph/mistral-7b-instruct-v0.2.Q4_K_M.gguf",
    temperature=0.1,
    max_tokens=2000,
    n_ctx=4096,
    top_p=0.95,
    verbose=False
)

# Define tool functions
def search_web(query: str) -> str:
    """Simulate web search."""
    return f"Found results for: {query}"

def calculate(expression: str) -> str:
    """Evaluate a mathematical expression."""
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {str(e)}"

def write_file(content: str) -> str:
    """Simulate writing content to a file."""
    return f"Successfully wrote: {content}"

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


In [12]:
class PlanAgent:
    def __init__(self, model: LlamaCpp):
        self.model = model
        
    def create_plan(self, query: str) -> List[Dict[str, Any]]:
        """Create a plan by breaking down the query into subtasks."""
        prompt = f"""<s>[INST] Break this task into exactly 2 simple steps:
        Task: {query}
        
        Create 2 separate tasks:
        1. First find/search for the required information
        2. Then perform the calculation or action on that information
        
        Format as a Python list with exactly 2 dictionaries:
        [
            {{"task_id": 1, "description": "Search for specific information", "status": "pending"}},
            {{"task_id": 2, "description": "Perform calculation with the found information", "status": "pending"}}
        ]
        
        Return only the Python list.[/INST]</s>"""
        
        try:
            response = self.model.invoke(prompt)
            # Clean and parse the response
            response_clean = response.strip().split('[')[1].split(']')[0]
            response_clean = f"[{response_clean}]"
            tasks = eval(response_clean)
            
            # Validate tasks
            if not tasks or len(tasks) == 0:
                raise ValueError("No tasks created")
                
            # Ensure proper task structure
            cleaned_tasks = []
            for i, task in enumerate(tasks, 1):
                cleaned_tasks.append({
                    "task_id": i,
                    "description": task.get("description", f"Step {i} of: {query}"),
                    "status": "pending"
                })
            return cleaned_tasks
            
        except Exception as e:
            # Fallback to basic task breakdown
            return [
                {
                    "task_id": 1,
                    "description": f"Search for information: {query}",
                    "status": "pending"
                },
                {
                    "task_id": 2,
                    "description": f"Process the found information",
                    "status": "pending"
                }
            ]

In [13]:
class ToolAgent:
    def __init__(self, tools: List[Tool], model: LlamaCpp):
        self.tools = tools
        self.model = model
        self.task_results = {}  # Store results for dependency handling
        
    def execute_task(self, task: Dict[str, Any]) -> Dict[str, Any]:
        """Execute a single task using available tools."""
        # Determine tool based on task type
        if "search" in task["description"].lower() or "find" in task["description"].lower():
            tool_name = "web_search"
        elif any(word in task["description"].lower() for word in ["calculate", "compute", "percent", "%"]):
            tool_name = "calculator"
        elif any(word in task["description"].lower() for word in ["save", "write", "store"]):
            tool_name = "file_writer"
        else:
            # Fallback to asking the model
            tool_prompt = f"""[INST] Which tool should be used for this task?
            TASK: {task['description']}
            Choose from: web_search, calculator, or file_writer
            Return only the tool name.[/INST]"""
            tool_name = self.model.invoke(tool_prompt).strip().lower()

        tool = next((t for t in self.tools if t.name.lower() == tool_name), None)
        
        if not tool:
            return {
                **task,
                "status": "failed",
                "error": f"No matching tool found: {tool_name}"
            }

        try:
            # Handle different tools appropriately
            if tool_name == "calculator":
                # For calculator, extract numbers from previous results if needed
                if task["task_id"] > 1:
                    prev_result = self.task_results.get(task["task_id"] - 1, "")
                    if "Found results for" in prev_result:
                        # Simulate getting actual population number (in real app, would parse from search)
                        population = "8336817"  # NYC population 2023 estimate
                        if "15%" in task["description"]:
                            tool_input = f"{population} * 0.15"
                        else:
                            tool_input = population
                    else:
                        tool_input = prev_result
                else:
                    tool_input = task["description"]
                    
            elif tool_name == "web_search":
                # For web search, use description directly
                tool_input = task["description"]
                
            else:  # file_writer
                tool_input = str(self.task_results.get(task["task_id"] - 1, "No data to write"))

            # Execute tool
            result = tool.run(tool_input)
            
            # Store result for potential future use
            self.task_results[task["task_id"]] = result
            
            return {
                **task,
                "status": "completed",
                "result": result,
                "tool_used": tool_name
            }
            
        except Exception as e:
            return {
                **task,
                "status": "failed",
                "error": str(e),
                "result": f"Failed to execute task: {str(e)}"
            }
    
    def reflect_on_result(self, task: Dict[str, Any]) -> str:
        """Provide feedback on task execution."""
        if task.get("status") == "completed":
            return f"Successfully completed task {task['task_id']} using {task.get('tool_used', 'unknown tool')}"
        else:
            return f"Failed to complete task {task['task_id']}: {task.get('error', 'unknown error')}"

In [14]:
class WorkflowState(TypedDict):
    query: str
    tasks: List[Dict[str, Any]]
    current_task_id: int
    completed: bool
    steps: int
    last_node: str

def create_workflow(tools: List[Tool], model: LlamaCpp) -> Graph:
    """Create a simplified workflow that executes tasks sequentially."""
    
    # Initialize agents with the local model
    plan_agent = PlanAgent(model)
    tool_agent = ToolAgent(tools, model)
    
    def plan(state: WorkflowState) -> WorkflowState:
        """Create initial plan if no tasks exist."""
        if not state["tasks"]:
            state["tasks"] = plan_agent.create_plan(state["query"])
            state["steps"] = 0
            state["last_node"] = "plan"
            state["current_task_id"] = 1  # Start with first task
        return state
    
    def execute(state: WorkflowState) -> WorkflowState:
        """Execute current task."""
        state["steps"] = state.get("steps", 0) + 1
        state["last_node"] = "execute"
        
        # Get current task
        current_task = next((t for t in state["tasks"] 
                           if t["task_id"] == state["current_task_id"]), None)
        
        if current_task and current_task["status"] == "pending":
            # Execute the task
            result = tool_agent.execute_task(current_task)
            
            # Update the task in state
            state["tasks"] = [
                result if t["task_id"] == current_task["task_id"] else t
                for t in state["tasks"]
            ]
            
            # Move to next task
            if state["current_task_id"] < len(state["tasks"]):
                state["current_task_id"] += 1
            else:
                state["completed"] = True
                
        return state
    
    def should_continue(state: WorkflowState) -> Tuple[bool, str]:
        """Determine next step in workflow."""
        # Stop if completed
        if state["completed"]:
            return False, "end"
            
        # Stop if too many steps
        if state["steps"] >= 10:
            state["completed"] = True
            return False, "end"
            
        # Initial planning
        if not state["tasks"]:
            return True, "plan"
        
        # Check if current task needs execution
        current_task = next((t for t in state["tasks"] 
                           if t["task_id"] == state["current_task_id"]), None)
        
        if current_task and current_task["status"] == "pending":
            return True, "execute"
        
        # Check if there are more tasks
        if state["current_task_id"] < len(state["tasks"]):
            return True, "execute"
        
        # No more tasks to execute
        state["completed"] = True
        return False, "end"
    
    # Create graph
    workflow = StateGraph(WorkflowState)
    
    # Add nodes
    workflow.add_node("plan", plan)
    workflow.add_node("execute", execute)
    
    # Set entry point
    workflow.set_entry_point("plan")
    
    # Add edges
    workflow.add_conditional_edges("plan", should_continue)
    workflow.add_conditional_edges("execute", should_continue)
    
    return workflow.compile()

def run_workflow(graph: Graph, query: str) -> Dict[str, Any]:
    """Run workflow with initial query."""
    initial_state: WorkflowState = {
        "query": query,
        "tasks": [],
        "current_task_id": 1,
        "completed": False,
        "steps": 0,
        "last_node": ""
    }
    return graph.invoke(initial_state)

In [15]:
# Create tools
tools = [
    Tool(
        name="web_search",
        func=search_web,
        description="Search the web for information."
    ),
    Tool(
        name="calculator",
        func=calculate,
        description="Calculate mathematical expressions."
    ),
    Tool(
        name="file_writer",
        func=write_file,
        description="Write content to a file."
    )
]

try:
    # Create workflow with local Mistral model
    workflow = create_workflow(tools, mistral)
    
    # Test query
    query = "Find the population of New York City and calculate 15% of it"
    
    # Run workflow
    result = run_workflow(workflow, query)
    
    print("\nWorkflow completed!")
    print("\nTasks and Results:")
    for task in result["tasks"]:
        print(f"\nTask {task['task_id']}:")
        print(f"Description: {task['description']}")
        print(f"Status: {task['status']}")
        if "result" in task:
            print(f"Result: {task['result']}")
            
except Exception as e:
    print(f"Error: {str(e)}")

C:\Users\DaysPC\anaconda3\envs\py310\lib\site-packages\llama_cpp\llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(



Workflow completed!

Tasks and Results:

Task 1:
Description: Search for the population of New York City
Status: completed
Result: Found results for: Search for the population of New York City

Task 2:
Description: Calculate 15% of the New York City population
Status: completed
Result: 1250522.55
